In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import time
import os

from google.cloud import storage


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def remove_users(df, lawyer_ids, name):
    filtered_df = df[~df[name].isin(lawyer_ids)]
    return filtered_df

def read_data_from_gcs(bucket_name, folder, filename, delimiter=','):
    start_time = time.time()  # Start measuring time
    blob = storage_client.get_bucket(bucket_name).blob(f'{folder}/{filename}')
    csv_data = blob.download_as_text()
    df = pd.read_csv(StringIO(csv_data), delimiter=delimiter, low_memory=False)
    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Read {filename} complete. Elapsed time: {elapsed_time:.2f} seconds")
    return df


In [ ]:
# Set path
relative_path = '../deep-flash-sa.json'
file_path = os.path.abspath(relative_path)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = file_path
storage_client = storage.Client()

In [ ]:
# Read branch_lawyer from local files
branch = pd.read_csv('branch_peradi_20240304.csv')
branch['email'] = branch['email'].str.lower()
branch.head()

,nama,email,branch_name
0,"Venny Romatua Damanik, S.H.",vennydamanik1972@gmail.com,Otto Hasibuan & Associates
1,"Guntur Perdamaian, S.H.",gunturperdamaian@gmail.com,PBH Peradi Pusat
2,"Dr. Christine Susanti, S.H., M.Hum.",chrislosant22@gmail.com,PBH Peradi Tangerang
3,"Donni Siagian, S.H.",donnisiagian@gmail.com,Otto Hasibuan & Associates
4,"Y.S. Parsiholan, S.H.",rollandlaw@gmail.com,PBH Peradi Pusat


In [ ]:
# Extract from consultation
filtered_consultations = df_consultations[['id', 'lawyer_id', 'skill_id', 'client_id', 'lawyer_approved', 'status', 'created_at', 'updated_at', 'matter', 'legal_basis', 'analysis', 'conclusion']].copy()
filtered_consultations['created_at'] = pd.to_datetime(filtered_consultations['created_at'])
filtered_consultations['updated_at'] = pd.to_datetime(filtered_consultations['updated_at'])

# Extract from rating
df_rating['created_at'] = pd.to_datetime(df_rating['created_at'])
df_rating['updated_at'] = pd.to_datetime(df_rating['updated_at'])

# Extract from rating
df_availability_instants['start_datetime'] = pd.to_datetime(df_availability_instants['start_datetime'])
df_availability_instants['end_datetime'] = pd.to_datetime(df_availability_instants['end_datetime'])

In [ ]:
filtered_consultations.head()

,id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,matter,legal_basis,analysis,conclusion
0,12,48,2,44,1,600,2023-05-02 11:09:56,2023-08-24 13:10:34,asdasd,asdasd,asdasd,asdad
1,16,48,2,43,1,600,2023-04-11 13:24:51,2023-08-11 13:39:00,asdasd,asdasd,asdasd,asdasd
2,17,48,2,43,1,600,2023-04-12 09:07:46,2023-04-12 09:42:59,asdads,asdasda,sdasd,asdasd
3,18,48,2,43,1,600,2023-04-12 11:08:56,2023-08-24 13:10:34,asdasd,asdasd,asdasd,asdasd
4,19,48,6,49,1,900,2023-04-12 12:53:06,2023-04-12 12:56:14,NaN,NaN,NaN,NaN


In [ ]:
# Clean testing data
lawyer_user = [36, 38, 48, 120, 192, 195]
client_user = [25, 42, 43, 44, 45, 49, 54, 56, 58, 62, 63, 66, 78, 85, 146, 157, 295, 959, 1609, 1655, 1821,
               2421, 2526, 2995, 3180, 4205, 5804, 5717, 5716, 7569, 7749, 7743, 11746]

filtered_consultations = remove_users(filtered_consultations, lawyer_user, 'lawyer_id')
filtered_consultations = remove_users(filtered_consultations, client_user, 'client_id')


df_rating = remove_users(df_rating, lawyer_user, 'lawyer_id')
df_availability_instants = remove_users(df_availability_instants, lawyer_user, 'lawyer_id')

In [ ]:
filtered_consultations

,id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,matter,legal_basis,analysis,conclusion
31,109,33,5,165,1,900,2023-05-16 14:29:04,2023-05-16 14:36:12,NaN,NaN,NaN,NaN
32,134,17,5,227,1,900,2023-05-19 10:45:17,2023-05-19 10:50:29,NaN,NaN,NaN,NaN
37,50,31,3,72,1,600,2023-05-06 20:26:54,2023-09-05 12:29:14,-,-,-,-
40,53,31,2,86,1,900,2023-05-09 10:35:37,2023-05-09 10:46:18,NaN,NaN,NaN,NaN
41,54,31,3,87,1,900,2023-05-09 10:55:36,2023-05-09 11:41:22,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15716,15864,59,5,14074,1,600,2024-02-04 20:13:00,2024-02-04 20:58:20,NaN,NaN,NaN,NaN
15717,15863,140,2,14092,1,600,2024-02-04 20:04:53,2024-02-04 20:16:58,NaN,NaN,NaN,NaN
15718,15861,64,4,14089,1,600,2024-02-04 19:00:17,2024-02-04 19:11:38,NaN,NaN,NaN,NaN
15719,15865,271,5,11692,0,333,2024-02-04 20:34:50,2024-02-04 20:35:46,NaN,NaN,NaN,NaN


In [ ]:
# Filter by month
date_start = '2024-01-01'
date_end = '2024-02-01'

consultations_within_range = filtered_consultations[(filtered_consultations['created_at'] >= date_start) & (filtered_consultations['created_at'] < date_end)]
df_rating_within_range = df_rating[(df_rating['created_at'] >= date_start) & (df_rating['created_at'] < date_end)]
df_availability_instants_with_range = df_availability_instants[(df_availability_instants['created_at'] >= date_start) & (df_availability_instants['created_at'] < date_end)]

In [ ]:
consultations_within_range.sort_values(by='created_at')

,id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,matter,legal_basis,analysis,conclusion
10979,11005,74,2,11157,1,600,2024-01-01 00:58:31,2024-01-01 01:24:50,NaN,NaN,NaN,NaN
10958,11006,140,2,11157,1,600,2024-01-01 01:30:23,2024-01-01 01:59:51,NaN,NaN,NaN,NaN
10962,11007,304,2,11157,0,900,2024-01-01 04:35:44,2024-01-01 04:38:45,NaN,NaN,NaN,NaN
10965,11008,304,2,11157,0,900,2024-01-01 04:39:09,2024-01-01 04:42:09,NaN,NaN,NaN,NaN
10952,11009,133,2,11157,1,600,2024-01-01 04:42:24,2024-01-01 05:10:31,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14981,15265,294,5,11517,1,600,2024-01-31 23:13:31,2024-01-31 23:25:40,NaN,NaN,NaN,NaN
14982,15266,101,4,13804,0,900,2024-01-31 23:14:04,2024-01-31 23:25:12,NaN,NaN,NaN,NaN
14997,15267,74,4,13804,0,900,2024-01-31 23:18:12,2024-01-31 23:21:13,NaN,NaN,NaN,NaN
14996,15268,74,4,13804,0,900,2024-01-31 23:18:47,2024-01-31 23:21:47,NaN,NaN,NaN,NaN


In [ ]:
# Filter columns
consultations_within_range = consultations_within_range[['id', 'lawyer_id', 'skill_id', 'client_id', 'lawyer_approved', 'status',
       'created_at', 'updated_at']]

KeyError: "['id'] not in index"

In [ ]:
# Filter by case done
consultations_within_range = consultations_within_range[consultations_within_range['status'] == 600]
consultations_within_range = consultations_within_range.rename(columns={'id':'consultation_id'})

In [ ]:
# Filter rating and rename column
df_rating_filtered = df_rating[['consultation_id', 'rating']].copy()
df_rating_filtered.tail()

In [ ]:
# Merge consultations with range
consultations_rating = consultations_within_range.merge(df_rating_filtered, on='consultation_id', how="left")
consultations_rating.tail(15)

,consultation_id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,rating
2115,15250,160,5,10262,1,600,2024-01-31 20:12:42,2024-01-31 20:27:27,NaN
2116,15111,313,2,13735,1,600,2024-01-31 10:34:56,2024-01-31 11:05:40,NaN
2117,15099,160,3,13728,1,600,2024-01-31 09:21:53,2024-01-31 09:54:09,5.0
2118,15108,294,5,13437,1,600,2024-01-31 10:27:23,2024-01-31 10:40:00,5.0
2119,15251,310,2,13426,1,600,2024-01-31 20:13:31,2024-01-31 20:44:14,5.0
2120,15203,59,3,13781,1,600,2024-01-31 16:49:35,2024-01-31 17:37:22,5.0
2121,15100,294,3,13711,1,600,2024-01-31 09:36:26,2024-01-31 09:45:28,5.0
2122,15121,317,3,13736,1,600,2024-01-31 10:40:56,2024-01-31 11:02:54,5.0
2123,15101,64,3,9461,1,600,2024-01-31 09:47:29,2024-01-31 10:17:06,5.0
2124,15109,222,3,13733,1,600,2024-01-31 10:34:41,2024-01-31 11:04:39,5.0


In [ ]:
#filter consultation summaries by columns
df_consultation_summaries = df_consultation_summaries[['consultation_id', 'matter', 'legal_basis', 'analysis', 'conclusion']]
df_consultation_summaries

,consultation_id,matter,legal_basis,analysis,conclusion
0,12,asdasd,asdasd,asdasd,asdad
1,16,asdasd,asdasd,asdasd,asdasd
2,17,asdads,asdasda,sdasd,asdasd
3,18,asdasd,asdasd,asdasd,asdasd
4,20,as,asdsa,asdasd,asdasd
...,...,...,...,...,...
9038,15864,Sore pak saya mau konsultasi masalah pernikaha...,Pasal 284 kuhp,Klien mengaku diancam oleh seseorang yg mengak...,Ssmoga peemasalahan segera selesai
9039,15866,Cerai gugat dan Pembuktian terkait anak hasil ...,Pasal 132 ayat (1) KHI ini,"dari perkara yang ibu jabarkan , yang saya tan...",jika di lampirkan dengan hasil tes DNA brati m...
9040,15854,Klien melakukan bisnis dan ada sisa pembayaran...,Kuhperdata ttg wanprestasi,"Sdh lama klien berbisnis dgn rekanya awal"" bis...",Saran saya sebelum mengajukan gugatan sebaikny...
9041,15855,pengancaman dan pemerasan melalui media sosial,Kitab Undang-Undang Hukum Pidana;\n\nUndang -U...,Share\n\n￼￼￼￼\n\n￼\n\n...>\n\nTeknologi\n\n>\n...,Share\n\n￼￼￼￼\n\n￼\n\n...>\n\nTeknologi\n\n>\n...


In [ ]:
consultations_rating = pd.merge(consultations_rating, df_consultation_summaries, on='consultation_id', how='left')

# Top Advocat

Top Advokat adalah penghargaan yang diberikan kepada 3 Advokat terbaik yang terpilih pada setiap bulannya, dengan ketentuan sebagai berikut.
- Advokat telah melakukan konsultasi dengan jumlah terbanyak dalam sebulan terakhir, minimal sebanyak 8 konsultasi
- Advokat telah mengisi seluruh ringkasan konsultasi yang telah diselesaikan, selambat-lambatnya 1x24 jam setelah konsultasi selesai
- Advokat mendapatkan rating yang didominasi oleh rating 5, dengan minimal rating 3
- Konsultasi dengan rating 1 dan 2 akan otomatis terdiskualifikasi, kecuali hal ini telah direvisi oleh Klien melalui proses identifikasi oleh Tim Perqara

https://docs.google.com/document/d/1J2i8pHPcgYY_7n12wucBws-aPPR7VRzJtrETKXqS2Nc/edit#heading=h.409mxmydrr07

In [ ]:
# Advocate with consultation more than 8
lawyer_appearances = consultations_rating.groupby('lawyer_id').size().reset_index(name='consultation_count')
lawyers_more_than_8 = lawyer_appearances[lawyer_appearances['consultation_count'] >= 8]
lawyers_more_than_8 = lawyers_more_than_8.sort_values('consultation_count', ascending=False)
lawyers_more_than_8 = lawyers_more_than_8['lawyer_id'].tolist()

consultations = consultations_rating[consultations_rating['lawyer_id'].isin(lawyers_more_than_8)]
print(consultations.shape)
consultations.head()

(1974, 13)


,consultation_id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,rating,matter,legal_basis,analysis,conclusion
0,11040,59,3,11168,1,600,2024-01-01 10:37:53,2024-01-01 11:08:49,5.0,Korban kekeraan seksual,Uu no 12 tahun 2022 tentang penghapusan kekera...,Klien bertanya bagaimana jika akan melaporkan ...,Dapat dilaporkan pidana dilengkapi bukti buktinya
1,11063,74,2,11181,1,600,2024-01-01 15:07:07,2024-01-01 15:26:31,NaN,klien sudah konsul tadi,ringkasan tetap sama,ringkasan tetap sama,ringkasan tetap sama
2,11081,88,2,11202,1,600,2024-01-01 19:39:03,2024-01-01 20:09:38,NaN,dicegat pemuda mambuk motor di tendak sampai j...,1.Kitab udang2 hukum pidana\n2.Undang -undang ...,dari permasalahan tersebut biasanya orang oran...,kesimpulan\nkejadian tersebut biasa sering dil...
3,11065,101,2,11188,1,600,2024-01-01 15:37:54,2024-01-01 16:09:00,3.0,adanya foto dan video klien yang diduga akan d...,undang-undang ITE,bahwa kasus yang klien miliki adalah adanya fo...,menyelesaikan secara kekeluargaan dan melakuka...
4,11082,101,3,10271,1,600,2024-01-01 19:49:47,2024-01-01 20:21:29,4.0,perjanjian kerja sama dalam kontrak kerja sesa...,perjanjian pasal 1320 KUHPerdata,bahwa adanya indikasi ketidakjujuran dalam pel...,untuk didampingi tim legal agar dapat menyelsa...


In [ ]:
# # Advocate with consultation more than 8
# lawyer_appearances = consultations_rating.groupby('lawyer_id').size().reset_index(name='consultation_count')
# lawyers_more_than_8 = lawyer_appearances[lawyer_appearances['consultation_count'] >= 8]
# lawyers_more_than_8 = lawyers_more_than_8.sort_values('consultation_count', ascending=False)
# lawyers_more_8 = lawyers_more_than_8['lawyer_id'].tolist()

# consultations = consultations_rating[consultations_rating['lawyer_id'].isin(lawyers_more_8)]
# print(consultations.shape)
# consultations.head()

In [ ]:
consultations_count = consultations['lawyer_id'].value_counts().reset_index().rename(columns={'index': 'lawyer_id', 'lawyer_id': 'consultation_count'})

In [ ]:
consultations_copy = consultations.copy()

In [ ]:
def calculate_summary_score(row):
    if any(pd.isnull(row[["matter", "legal_basis", "analysis", "conclusion"]])):
        return 0
    return 2.5

consultations_copy['consultation_score'] = 2.5
consultations_copy["summary_score"] = consultations_copy.apply(calculate_summary_score, axis=1)

In [ ]:
# Calculate rating score
rating_mapping = {5: 5, 4: 3, 3: 1, 2: -10, 1: -15}
consultations_copy["rating_score"] = consultations_copy["rating"].map(rating_mapping)
consultations_copy["rating_score"].fillna(0, inplace=True)  # Fill NaN with 0
consultations_copy.head()

,consultation_id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,rating,matter,legal_basis,analysis,conclusion,consultation_score,summary_score,rating_score
0,11040,59,3,11168,1,600,2024-01-01 10:37:53,2024-01-01 11:08:49,5.0,Korban kekeraan seksual,Uu no 12 tahun 2022 tentang penghapusan kekera...,Klien bertanya bagaimana jika akan melaporkan ...,Dapat dilaporkan pidana dilengkapi bukti buktinya,2.5,2.5,5.0
1,11063,74,2,11181,1,600,2024-01-01 15:07:07,2024-01-01 15:26:31,NaN,klien sudah konsul tadi,ringkasan tetap sama,ringkasan tetap sama,ringkasan tetap sama,2.5,2.5,0.0
2,11081,88,2,11202,1,600,2024-01-01 19:39:03,2024-01-01 20:09:38,NaN,dicegat pemuda mambuk motor di tendak sampai j...,1.Kitab udang2 hukum pidana\n2.Undang -undang ...,dari permasalahan tersebut biasanya orang oran...,kesimpulan\nkejadian tersebut biasa sering dil...,2.5,2.5,0.0
3,11065,101,2,11188,1,600,2024-01-01 15:37:54,2024-01-01 16:09:00,3.0,adanya foto dan video klien yang diduga akan d...,undang-undang ITE,bahwa kasus yang klien miliki adalah adanya fo...,menyelesaikan secara kekeluargaan dan melakuka...,2.5,2.5,1.0
4,11082,101,3,10271,1,600,2024-01-01 19:49:47,2024-01-01 20:21:29,4.0,perjanjian kerja sama dalam kontrak kerja sesa...,perjanjian pasal 1320 KUHPerdata,bahwa adanya indikasi ketidakjujuran dalam pel...,untuk didampingi tim legal agar dapat menyelsa...,2.5,2.5,3.0


In [ ]:
consultations_score = consultations_copy[['lawyer_id', 'consultation_score','summary_score', 'rating_score']]
sum_by_lawyer_id = consultations_score.groupby('lawyer_id').sum().reset_index()
sum_by_lawyer_id['total_score'] =sum_by_lawyer_id['consultation_score'] + sum_by_lawyer_id['summary_score'] + sum_by_lawyer_id['rating_score']
sum_by_lawyer_id_sorted = sum_by_lawyer_id.sort_values('total_score', ascending=False)
sum_by_lawyer_id_sorted

,lawyer_id,consultation_score,summary_score,rating_score,total_score
13,140,622.5,622.5,616.0,1861.0
5,74,680.0,680.0,450.0,1810.0
31,294,440.0,440.0,579.0,1459.0
1,59,395.0,395.0,392.0,1182.0
14,160,282.5,282.5,313.0,878.0
9,101,275.0,275.0,205.0,755.0
26,270,205.0,205.0,245.0,655.0
7,79,170.0,170.0,220.0,560.0
24,254,180.0,180.0,183.0,543.0
35,309,150.0,150.0,226.0,526.0


In [ ]:
top_lawyer = pd.merge(sum_by_lawyer_id_sorted, consultations_count, on='lawyer_id', how='left')

In [ ]:
df_lawyer_score = top_lawyer[['lawyer_id', 'consultation_count', 'total_score']]
df_lawyer_score

,lawyer_id,consultation_count,total_score
0,140,249,1861.0
1,74,272,1810.0
2,294,176,1459.0
3,59,158,1182.0
4,160,113,878.0
5,101,110,755.0
6,270,82,655.0
7,79,68,560.0
8,254,72,543.0
9,309,60,526.0


In [ ]:
lawyer_id_name = df_lawyers[['id', 'slug']].rename(columns={'id':'lawyer_id', 'slug':'lawyer_name'})

In [ ]:
final_top_advocate = pd.merge(df_lawyer_score, lawyer_id_name, on='lawyer_id', how='left')

In [ ]:
final_top_advocate

,lawyer_id,consultation_count,total_score,lawyer_name
0,140,249,1861.0,"Muhammad Celvin Alfarizy, S.H."
1,74,272,1810.0,"Fanizal Maulana, S.H."
2,294,176,1459.0,"Genuari Waruwu, S.H."
3,59,158,1182.0,"Burhan Habibi, S.H."
4,160,113,878.0,"Sepriyanto, S.H., CPM."
5,101,110,755.0,"Dr. Desi Purnani Adam, S.H., M.H."
6,270,82,655.0,"Syarif Hidayatullah, S.H., M.H."
7,79,68,560.0,"Rhony B.S.P. Hamsah, S.H."
8,254,72,543.0,"Abdur Rozzak Harahap, S.H."
9,309,60,526.0,"Rezki Februarianto, S.H."


In [ ]:
final_top_advocate[['lawyer_name', 'consultation_count', 'total_score']][:10]

,lawyer_name,consultation_count,total_score
0,"Muhammad Celvin Alfarizy, S.H.",249,1861.0
1,"Fanizal Maulana, S.H.",272,1810.0
2,"Genuari Waruwu, S.H.",176,1459.0
3,"Burhan Habibi, S.H.",158,1182.0
4,"Sepriyanto, S.H., CPM.",113,878.0
5,"Dr. Desi Purnani Adam, S.H., M.H.",110,755.0
6,"Syarif Hidayatullah, S.H., M.H.",82,655.0
7,"Rhony B.S.P. Hamsah, S.H.",68,560.0
8,"Abdur Rozzak Harahap, S.H.",72,543.0
9,"Rezki Februarianto, S.H.",60,526.0


In [ ]:
# final_top_advocate.to_csv('top_advocate.csv', index=False)

In [ ]:
top = sum_by_lawyer_id_sorted['lawyer_id'][:10].values
for rank, lawyer_id in enumerate(top, start=1):
    lawyer_slug = df_lawyers.loc[df_lawyers['id'] == lawyer_id, 'slug'].values[0]
#     print(f"Top {rank}: {lawyer_slug} (with id: {lawyer_id})")
    print(f"{lawyer_slug}")

Muhammad Celvin Alfarizy, S.H.
Fanizal Maulana, S.H.
Genuari Waruwu, S.H.
Burhan Habibi, S.H.
Sepriyanto, S.H., CPM.
Dr. Desi Purnani Adam, S.H., M.H.
Syarif Hidayatullah, S.H., M.H.
Rhony B.S.P. Hamsah, S.H.
Abdur Rozzak Harahap, S.H.
Rezki Februarianto, S.H.


In [ ]:
# consultations_within_range_clean['time_difference'] = consultations_within_range_clean['updated_at'] - consultations_within_range_clean['created_at'].copy()
# result = consultations_within_range_clean[consultations_within_range_clean['time_difference'] > timedelta(days=1)]
# result

# Top Instansi
Top Instansi adalah penghargaan yang diberikan kepada 3 Cabang Instansi terbaik yang terpilih pada setiap bulannya, dengan ketentuan sebagai berikut.
- Cabang Instansi dengan total durasi online Advokat terbanyak dalam sebulan terakhir
- Cabang Instansi dengan jumlah Advokat terbanyak dalam memberikan konsultasi sebulan terakhir
- Cabang Instansi dengan nilai rata-rata rating tertinggi untuk Advokat, dengan minimal rating adalah rating 3

https://docs.google.com/document/d/1J2i8pHPcgYY_7n12wucBws-aPPR7VRzJtrETKXqS2Nc/edit#heading=h.v08revnqy5i

## Indikator Penilaian Top Instansi
Indikator Skor Penilaian untuk Top Instansi dihitung dari akumulasi skor durasi Online Advokat dari Cabang Instansi dan skor Top Advokat. Setiap cabang Instansi akan mendapat 1 poin untuk durasi online 5 jam dari masing-masing Advokat dan berlaku kelipatannya, sebagai berikut:

Durasi Online Advokat
Bobot Nilai (Poin)
- 5 Jam -> 1 poin

In [ ]:
df_time_online = df_availability_instants_with_range[['lawyer_id', 'start_datetime', 'end_datetime']].copy()
df_time_online['duration'] = (df_time_online['end_datetime'] - df_time_online['start_datetime']).astype('timedelta64[m]') / 60
df_time_online.head()

,lawyer_id,start_datetime,end_datetime,duration
10282,88,2024-01-01 00:13:35,2024-01-01 00:15:27,0.016667
10283,140,2024-01-01 00:37:39,2024-01-01 02:04:52,1.450000
10284,101,2024-01-01 04:48:43,2024-01-01 07:11:20,2.366667
10285,101,2024-01-01 07:39:45,2024-01-01 08:26:00,0.766667
10286,304,2024-01-01 02:57:56,2024-01-01 09:51:18,6.883333


In [ ]:
total_duration = df_time_online.groupby('lawyer_id')['duration'].sum().reset_index()
total_duration = total_duration.sort_values('duration', ascending=False)
total_duration['duration'] = (total_duration['duration'] + 0.5).astype(int)
total_duration.head()

,lawyer_id,duration
17,74,535
43,140,476
98,294,288
82,254,287
8,59,261


In [ ]:
# Advocate with consultation
lawyer_appearances = consultations_rating.groupby('lawyer_id').size().reset_index(name='consultation_count')
# lawyers_more_than_8 = lawyer_appearances[lawyer_appearances['consultation_count'] >= 8]
lawyer_appearances = lawyer_appearances.sort_values('consultation_count', ascending=False)
lawyer_appearances = lawyer_appearances['lawyer_id'].tolist()

consultations = consultations_rating[consultations_rating['lawyer_id'].isin(lawyer_appearances)]
consultations_count = consultations['lawyer_id'].value_counts().reset_index().rename(columns={'index': 'lawyer_id', 'lawyer_id': 'consultation_count'})

In [ ]:
consultations_copy = consultations.copy()
def calculate_summary_score(row):
    if any(pd.isnull(row[["matter", "legal_basis", "analysis", "conclusion"]])):
        return 0
    return 2.5

consultations_copy['consultation_score'] = 2.5
consultations_copy["summary_score"] = consultations_copy.apply(calculate_summary_score, axis=1)

# Calculate rating score
rating_mapping = {5: 5, 4: 3, 3: 1, 2: -10, 1: -15}
consultations_copy["rating_score"] = consultations_copy["rating"].map(rating_mapping)
consultations_copy["rating_score"].fillna(0, inplace=True)  # Fill NaN with 0
consultations_copy.head()

,consultation_id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,rating,matter,legal_basis,analysis,conclusion,consultation_score,summary_score,rating_score
0,11040,59,3,11168,1,600,2024-01-01 10:37:53,2024-01-01 11:08:49,5.0,Korban kekeraan seksual,Uu no 12 tahun 2022 tentang penghapusan kekera...,Klien bertanya bagaimana jika akan melaporkan ...,Dapat dilaporkan pidana dilengkapi bukti buktinya,2.5,2.5,5.0
1,11063,74,2,11181,1,600,2024-01-01 15:07:07,2024-01-01 15:26:31,NaN,klien sudah konsul tadi,ringkasan tetap sama,ringkasan tetap sama,ringkasan tetap sama,2.5,2.5,0.0
2,11081,88,2,11202,1,600,2024-01-01 19:39:03,2024-01-01 20:09:38,NaN,dicegat pemuda mambuk motor di tendak sampai j...,1.Kitab udang2 hukum pidana\n2.Undang -undang ...,dari permasalahan tersebut biasanya orang oran...,kesimpulan\nkejadian tersebut biasa sering dil...,2.5,2.5,0.0
3,11065,101,2,11188,1,600,2024-01-01 15:37:54,2024-01-01 16:09:00,3.0,adanya foto dan video klien yang diduga akan d...,undang-undang ITE,bahwa kasus yang klien miliki adalah adanya fo...,menyelesaikan secara kekeluargaan dan melakuka...,2.5,2.5,1.0
4,11082,101,3,10271,1,600,2024-01-01 19:49:47,2024-01-01 20:21:29,4.0,perjanjian kerja sama dalam kontrak kerja sesa...,perjanjian pasal 1320 KUHPerdata,bahwa adanya indikasi ketidakjujuran dalam pel...,untuk didampingi tim legal agar dapat menyelsa...,2.5,2.5,3.0


In [ ]:
consultations_copy

,consultation_id,lawyer_id,skill_id,client_id,lawyer_approved,status,created_at,updated_at,rating,matter,legal_basis,analysis,conclusion,consultation_score,summary_score,rating_score
0,11040,59,3,11168,1,600,2024-01-01 10:37:53,2024-01-01 11:08:49,5.0,Korban kekeraan seksual,Uu no 12 tahun 2022 tentang penghapusan kekera...,Klien bertanya bagaimana jika akan melaporkan ...,Dapat dilaporkan pidana dilengkapi bukti buktinya,2.5,2.5,5.0
1,11063,74,2,11181,1,600,2024-01-01 15:07:07,2024-01-01 15:26:31,NaN,klien sudah konsul tadi,ringkasan tetap sama,ringkasan tetap sama,ringkasan tetap sama,2.5,2.5,0.0
2,11081,88,2,11202,1,600,2024-01-01 19:39:03,2024-01-01 20:09:38,NaN,dicegat pemuda mambuk motor di tendak sampai j...,1.Kitab udang2 hukum pidana\n2.Undang -undang ...,dari permasalahan tersebut biasanya orang oran...,kesimpulan\nkejadian tersebut biasa sering dil...,2.5,2.5,0.0
3,11065,101,2,11188,1,600,2024-01-01 15:37:54,2024-01-01 16:09:00,3.0,adanya foto dan video klien yang diduga akan d...,undang-undang ITE,bahwa kasus yang klien miliki adalah adanya fo...,menyelesaikan secara kekeluargaan dan melakuka...,2.5,2.5,1.0
4,11082,101,3,10271,1,600,2024-01-01 19:49:47,2024-01-01 20:21:29,4.0,perjanjian kerja sama dalam kontrak kerja sesa...,perjanjian pasal 1320 KUHPerdata,bahwa adanya indikasi ketidakjujuran dalam pel...,untuk didampingi tim legal agar dapat menyelsa...,2.5,2.5,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,15104,207,4,13732,1,600,2024-01-31 10:02:03,2024-01-31 10:32:21,NaN,Saya adalah dosen di salah satu perguruan ting...,Peraturan Menteri Pendidikan dan Kebudayaan Re...,"Prosedurnya dijalani saja pak, dgn membuat sur...",Surat dari kampus anda ke kemendikbud untuk me...,2.5,2.5,0.0
2126,15106,294,4,12321,1,600,2024-01-31 10:09:17,2024-01-31 10:17:16,NaN,"Di PHK ditahun 2019, namun pesangon hanya diba...",Undang Undang Nomor 13 Tahun 2003 tentang Kete...,"Jika Pesangon belum dibyarkan full, mka lakuka...",Saya sarankan untuk melakukan perundingan terl...,2.5,2.5,0.0
2127,15171,243,2,13763,1,600,2024-01-31 15:09:13,2024-01-31 15:39:37,NaN,melihat/mengintip wanita mandi lewat ventilas...,"Kuhp tentang kejahatan Psl 281 ayat (1), kejah...",mengintip orang lain mandi bisa bepotensi masu...,Setiap orang masuk ke ngintip orang mandi deng...,2.5,2.5,0.0
2128,15193,74,5,13779,1,600,2024-01-31 16:26:03,2024-01-31 16:56:36,5.0,menanyakan mengenai perceraian,UU no 1 tahun 1974 perkawinan,jika mau mengajukan cerai gaib urus terlebih d...,mohon di rating dan semoga segera teratasi,2.5,2.5,5.0


In [ ]:
consultations_score = consultations_copy[['lawyer_id', 'consultation_score','summary_score', 'rating_score']]
sum_by_lawyer_id = consultations_score.groupby('lawyer_id').sum().reset_index()
sum_by_lawyer_id['total_score'] =sum_by_lawyer_id['consultation_score'] + sum_by_lawyer_id['summary_score'] + sum_by_lawyer_id['rating_score']
sum_by_lawyer_id_sorted = sum_by_lawyer_id.sort_values('total_score', ascending=False)
sum_by_lawyer_id_sorted

,lawyer_id,consultation_score,summary_score,rating_score,total_score
36,140,622.5,622.5,616.0,1861.0
14,74,680.0,680.0,450.0,1810.0
75,294,440.0,440.0,579.0,1459.0
6,59,395.0,395.0,392.0,1182.0
40,160,282.5,282.5,313.0,878.0
...,...,...,...,...,...
54,221,5.0,0.0,0.0,5.0
26,106,2.5,2.5,0.0,5.0
56,226,2.5,2.5,0.0,5.0
49,202,2.5,0.0,1.0,3.5


In [ ]:
lawyer_score = pd.merge(sum_by_lawyer_id_sorted, consultations_count, on='lawyer_id', how='left')

In [ ]:
#WARNING
df_merged_lawyer = pd.merge(total_duration, lawyer_score, on='lawyer_id', how='left')
df_merged_lawyer.head()

,lawyer_id,duration,consultation_score,summary_score,rating_score,total_score,consultation_count
0,74,535,680.0,680.0,450.0,1810.0,272.0
1,140,476,622.5,622.5,616.0,1861.0,249.0
2,294,288,440.0,440.0,579.0,1459.0,176.0
3,254,287,180.0,180.0,183.0,543.0,72.0
4,59,261,395.0,395.0,392.0,1182.0,158.0


In [ ]:
# Extract necessary columns from df_lawyers and rename them
df_lawyers_prep = df_lawyers.copy()
df_lawyers_prep.rename(columns={'id': 'lawyer_id', 'slug': 'lawyer_name'}, inplace=True)
lawyers = df_lawyers_prep[['lawyer_id', 'user_id', 'lawyer_name', 'agency_name', 'address', 'city_id']].copy()

# Extract necessary columns from df_users and rename them
user_emails = df_users[['id', 'email']].copy()
user_emails.rename(columns={'id': 'user_id'}, inplace=True)
# Merge lawyers and user_emails DataFrames based on user_id
merged_df = merge_dataframes(lawyers, user_emails, on='user_id')

In [ ]:
branch['email'] = branch['email'].astype(str)
merged_df['agency_name'] = merged_df['agency_name'].replace('PBH DPN Peradi', 'PBH Peradi')
merged_df['agency_name'] = merged_df['agency_name'].replace('PBH PERADI', 'PBH Peradi')
merged_df['agency_name'] = merged_df['agency_name'].replace('PBH DPC Peradi', 'PBH Peradi')

# Potential Error
merged_df_final = pd.merge(merged_df, branch, on='email', how='left')
branch_lawyers = merged_df_final[['branch_name', 'lawyer_id']]

In [ ]:
merged_df_final[200:250]

,lawyer_id,user_id,lawyer_name,agency_name,address,city_id,email,nama,branch_name
200,264,5576,"Zagky Drajat, S.H.",PBH Peradi,,3204.0,zd.lawfirm1184@gmail.com,"Zagky Drajat, S.H.",PBH Peradi Bale Bandung
201,259,4434,"I Komang Darmayasa, S.H., M.H.",YLC Peradi,,5104.0,ikm_darmayasa@yahoo.co.id,"I Komang Darmayasa, S.H., M.H.",YLC DENPASAR
202,260,4830,"Aswar, S.H., M.H.",YLC Peradi,"Furia Puskopad Blok D RT 003 / RW 005, Wahino,...",9403.0,aswar.jpa@gmail.com,"Aswar, S.H., M.H.",YLC JAYAPURA
203,265,5577,"Raju Diagunsyah, S.H., M.H.",YLC Peradi,,1671.0,rajudiagunsyah@gmail.com,"Raju Diagunsyah, S.H., M.H.",YLC PALEMBANG
204,267,5907,"Supriyanti, S.H., M.H., C.Me",PBH Peradi,,3603.0,yanti.syahroni@gmail.com,"Supriyanti, S.H., M.H., C.Me",PBH Peradi Tangerang
205,268,6011,"Camelia Ahmad, S.H., M.H.",YLC Peradi,,3276.0,camelia.ahmad@dslc.law,"Camelia Ahmad, S.H., M.H.",Dwinanto Strategic Legal Consultant (DSLC)
206,48,61,Pringgo Jr,,Senopati,3171.0,pringgo@perqara.com,NaN,NaN
207,59,330,"Burhan Habibi, S.H.",PBH Peradi,"Jl. Mayor Sujadi 30 A RT 004 / RW 002, Jepun, ...",3507.0,boerhanhabibi@gmail.com,"Burhan Habibi, S.H.",PBH Peradi Malang
208,272,6869,"Reza Afthoni, S.H.",YLC Peradi,"Blok Katimaa, RT/RW 016/004, Kel. Ciawi, Kec. ...",3603.0,rezaafthoni@gmail.com,"Reza Afthoni, S.H.",YLC Peradi Tangerang
209,273,6870,"Fery Zuriansyah, S.H.",PBH Peradi,"Jl Tugu Karya III No. 52, RT/RW 002/010, Kel. ...",3603.0,feryzuriansyah@gmail.com,"Fery Zuriansyah, S.H.",PBH Peradi Tangerang


In [ ]:
# Lawyer A.N Roby Alamsyah, S.H. gaada email dan branch di database, tapi di sheets BD ada

# typo
# rumintangnaibHo365@gmail.com
# rumintangnaibaho365@gmail.com

# typo
# lalutanggasatria@gmail.com
# laluranggasatria@gmail.com

# beda domain
# eveline.green@yahoo.id
# eveline.green@yahoo.co.id

# beda email
# advokat.panji@gmail.com
# advokat.bakhtiar@gmail.com

# beda email
# yodikristianto@gmail.com
# yodikristianto26@gmail.com

# typo
# desi_grameza@yahoo.com
# desi_gramezha@yahoo.com

# Nama bang yakup gaada di sheets bd, jadinya branchnya ngga kebaca

# Ngga kebaca
# Andrian Febrianto, S.H.,M.H.,C.L.A
# Libet Astoyo, S.H.

In [ ]:
# merged_df_final[merged_df_final['user_id'] == 3428]
# consultations_within_range[consultations_within_range['lawyer_id'] == 262]
# df_lawyers[df_lawyers['id'] == 262]
# df_users[df_users['id'] == 5210]

In [ ]:
# merged_df_final['agency_name'].value_counts()
# merged_df['agency_name'].str.upper().value_counts()

In [ ]:
# df_lawyers[df_lawyers['id'] == 221][['slug', 'agency_name']]
# # 101, 160

In [ ]:
# final[final['branch_name'] == 'PBH Peradi Tangerang']

In [ ]:
final = pd.merge(df_merged_lawyer, branch_lawyers, on='lawyer_id', how='left')
final.sort_values(by='consultation_score', ascending=False).head(10)

,lawyer_id,duration,consultation_score,summary_score,rating_score,total_score,consultation_count,branch_name
0,74,535,680.0,680.0,450.0,1810.0,272.0,PBH Peradi Malang
1,140,476,622.5,622.5,616.0,1861.0,249.0,PBH Peradi Malang
2,294,288,440.0,440.0,579.0,1459.0,176.0,NaN
4,59,261,395.0,395.0,392.0,1182.0,158.0,PBH Peradi Malang
10,160,139,282.5,282.5,313.0,878.0,113.0,PBH Peradi Cikarang
6,101,188,275.0,275.0,205.0,755.0,110.0,PBH Peradi Denpasar
7,270,180,205.0,205.0,245.0,655.0,82.0,PBH Peradi Tangerang
3,254,287,180.0,180.0,183.0,543.0,72.0,YLC MEDAN
8,271,160,175.0,175.0,156.0,506.0,70.0,NaN
21,79,65,170.0,170.0,220.0,560.0,68.0,PBH Peradi Jember


In [ ]:
final[100:]

,lawyer_id,duration,consultation_score,summary_score,rating_score,total_score,consultation_count,branch_name
100,216,1,NaN,NaN,NaN,NaN,NaN,YLC SUNGGUMINASA
101,208,1,NaN,NaN,NaN,NaN,NaN,YLC BANJARMASIN
102,228,0,NaN,NaN,NaN,NaN,NaN,YLC BANDAR LAMPUNG
103,190,0,NaN,NaN,NaN,NaN,NaN,PBH Peradi Cikarang
104,31,0,2.5,0.0,0.0,2.5,1.0,PBH Peradi Pusat
105,153,0,NaN,NaN,NaN,NaN,NaN,PBH Peradi Cikarang
106,181,0,NaN,NaN,NaN,NaN,NaN,PBH Peradi Banggai
107,202,0,2.5,0.0,1.0,3.5,1.0,YLC TEGAL
108,301,0,NaN,NaN,NaN,NaN,NaN,NaN
109,50,0,NaN,NaN,NaN,NaN,NaN,PBH Peradi Ruteng


In [ ]:
# Get consultation by branch
consultation_branch = final.groupby('branch_name')['consultation_count'].sum().reset_index().sort_values(
    by='consultation_count', ascending=False)
consultation_branch.head()

,branch_name,consultation_count
27,PBH Peradi Malang,746.0
11,PBH Peradi Cikarang,134.0
12,PBH Peradi Denpasar,111.0
36,PBH Peradi Tangerang,99.0
46,YLC MEDAN,72.0


In [ ]:
result = final.groupby('branch_name').agg({'duration': 'sum', 'total_score': 'sum'}).reset_index()
result.head(10)

,branch_name,duration,total_score
0,Otto Hasibuan & Associates,2,15.0
1,PBH DPN Peradi,0,0.0
2,PBH Peradi Bale Bandung,3,25.0
3,PBH Peradi Balikpapan,107,478.0
4,PBH Peradi Bandung,2,25.0
5,PBH Peradi Banggai,0,0.0
6,PBH Peradi Banjarmasin,1,10.0
7,PBH Peradi Bengkulu,0,0.0
8,PBH Peradi Boyolali,9,0.0
9,PBH Peradi Bungo Tebo,8,15.0


In [ ]:
sorted_result = result.sort_values(by=['total_score', 'duration'], ascending=[False, True]).reset_index(drop=True)
sorted_result[:10]

,branch_name,duration,total_score
0,PBH Peradi Malang,1581,5389.0
1,PBH Peradi Cikarang,182,1053.0
2,PBH Peradi Tangerang,211,771.0
3,PBH Peradi Denpasar,189,765.0
4,PBH Peradi Jember,65,560.0
5,YLC MEDAN,287,543.0
6,PBH Peradi Balikpapan,107,478.0
7,YLC KEPANJEN,94,348.0
8,YLC DEPOK,87,295.0
9,PBH Peradi Ende,253,275.0


In [ ]:
sorted_result['duration_score'] = sorted_result['duration'] / 5
sorted_result['final_score'] = sorted_result['total_score'] + sorted_result['duration_score']
# sorted_result.sort_values('final_score', ascending=False)

In [ ]:
sorted_result_final = sorted_result.sort_values('final_score', ascending=False)
# sorted_result_final.to_csv('top_agency.csv', index=False)

In [ ]:
final_result = pd.merge(sorted_result_final[['branch_name', 'final_score']], consultation_branch, on='branch_name', how='left')

In [ ]:
# final_result.to_csv('top_agency.csv', index=False)

In [ ]:
final_result[['branch_name', 'consultation_count', 'final_score']][:10]

,branch_name,consultation_count,final_score
0,PBH Peradi Malang,746.0,5705.2
1,PBH Peradi Cikarang,134.0,1089.4
2,PBH Peradi Tangerang,99.0,813.2
3,PBH Peradi Denpasar,111.0,802.8
4,YLC MEDAN,72.0,600.4
5,PBH Peradi Jember,68.0,573.0
6,PBH Peradi Balikpapan,58.0,499.4
7,YLC KEPANJEN,43.0,366.8
8,PBH Peradi Ende,39.0,325.6
9,YLC DEPOK,38.0,312.4


In [ ]:
final_result[['branch_name', 'consultation_count', 'final_score']][:49]

,branch_name,consultation_count,final_score
0,PBH Peradi Malang,746.0,5705.2
1,PBH Peradi Cikarang,134.0,1089.4
2,PBH Peradi Tangerang,99.0,813.2
3,PBH Peradi Denpasar,111.0,802.8
4,YLC MEDAN,72.0,600.4
5,PBH Peradi Jember,68.0,573.0
6,PBH Peradi Balikpapan,58.0,499.4
7,YLC KEPANJEN,43.0,366.8
8,PBH Peradi Ende,39.0,325.6
9,YLC DEPOK,38.0,312.4


# Top Agency

Top Instansi adalah penghargaan yang diberikan kepada 3 Cabang Instansi terbaik yang terpilih pada setiap bulannya, dengan ketentuan sebagai berikut.
- Cabang Instansi dengan total durasi online Advokat terbanyak dalam sebulan terakhir
- Cabang Instansi dengan jumlah Advokat terbanyak dalam memberikan konsultasi sebulan terakhir
- Cabang Instansi dengan nilai rata-rata rating tertinggi untuk Advokat, dengan minimal rating adalah rating 3
